In [3]:
suppressPackageStartupMessages({
    library("edgeR")
    library("stringr")
    library("limma")
    library("RColorBrewer")
    library("Glimma")
    library("dplyr")
    library("clusterProfiler")
    library("org.Hs.eg.db")
})

In [4]:
stage_order = make.names( c('Fetal', 'Neonatal', 'Infancy', 'Childhood', 'Adolescence', 'Adult'))
stage_order

[1] "Fetal"       "Neonatal"    "Infancy"     "Childhood"   "Adolescence"
[6] "Adult"

### Read in all dev-traj files

In [9]:
dt_file_list = list.files( "../data/limma-voom", full.names=TRUE, pattern='major-dev-traj')
x <- dt_file_list
a <- "obs-cts"
obs_mk <- sapply(x, function(string) {all(Vectorize(grepl)(pattern = a, x = string))})
obs_files <- dt_file_list[obs_mk]
blk_files <- dt_file_list[!obs_mk]

In [13]:
# make sure observation and bulk files match in number
length( obs_files)==length( blk_files)

[1] TRUE

In [15]:
obs_files

[1] "../data/limma-voom/major-dev-traj_Astro_obs-cts_min10.csv"       
 [2] "../data/limma-voom/major-dev-traj_ID2_obs-cts_min10.csv"         
 [3] "../data/limma-voom/major-dev-traj_L2-3_CUX2_obs-cts_min10.csv"   
 [4] "../data/limma-voom/major-dev-traj_L4_RORB_obs-cts_min10.csv"     
 [5] "../data/limma-voom/major-dev-traj_L5-6_THEMIS_obs-cts_min10.csv" 
 [6] "../data/limma-voom/major-dev-traj_L5-6_TLE4_obs-cts_min10.csv"   
 [7] "../data/limma-voom/major-dev-traj_LAMP5_NOS1_obs-cts_min10.csv"  
 [8] "../data/limma-voom/major-dev-traj_Micro_obs-cts_min10.csv"       
 [9] "../data/limma-voom/major-dev-traj_Oligo_obs-cts_min10.csv"       
[10] "../data/limma-voom/major-dev-traj_OPC_obs-cts_min10.csv"         
[11] "../data/limma-voom/major-dev-traj_Poor-Quality_obs-cts_min10.csv"
[12] "../data/limma-voom/major-dev-traj_PV_obs-cts_min10.csv"          
[13] "../data/limma-voom/major-dev-traj_PV_SCUBE3_obs-cts_min10.csv"   
[14] "../data/limma-voom/major-dev-traj_SST_obs-cts_min10.csv"         
[15] "../data/limma-voom/major-dev-traj_Vas_obs-cts_min10.csv"         
[16] "../data/limma-voom/major-dev-traj_VIP_obs-cts_min10.csv"

In [28]:
for (itr in 1:length(obs_files)){
    # get files for each traj-dev in list of files
    obs_fl_itr <- obs_files[itr]
    blk_fl_itr <- blk_files[itr]
    counts <- read.delim( blk_fl_itr, row.names=1, sep=',')
    obs    <- read.delim( obs_fl_itr, row.names=1, sep=',')
    #####################################
    # get traj-dev name for output naming
    splt_dt_nm <- sapply( obs_fl_itr, function(x) 
        {strsplit( x, "_obs-cts_min10", fixed=TRUE)}[[1]])
    # get traj-dev name
    splt_dt_nm2 <- sapply( splt_dt_nm, function(x) 
        {strsplit( x, "major-dev-traj_", fixed=TRUE)}[[1]][c(2)])
    dt_nm <- paste( '../data/limma-voom/glimma_dev-traj_', splt_dt_nm2[1],  sep="")
    print( dt_nm)
    #####################################
    # create edgeR DGE object
    x <- DGEList( counts)
    rownames( obs) <- make.names( rownames( obs))      
    group <- as.factor( obs$stage_id)
    x$samples$group <- group
    # Rename all levels, by name
    levels(group) <- list('Fetal'='Fetal','Neonatal'='Neonatal','Infancy'='Infancy','Childhood'='Childhood',
                           'Adolescence'='Adolescence','Adult'='Adult')

    # add all wanted features to x$samples
    feats_oi <- c( 'chem', 'Sex', 'Library.Prep.Lot')
    for (feat_itr in feats_oi){
        fact <- as.factor( obs[feat_itr])
        x$samples[feat_itr] <- obs[feat_itr]
    }
    x$samples[,feats_oi] <- mutate_all( x$samples[,feats_oi], as.character)     
    # control for lot 2 only, all others show no effects
    prep_lot <- x$samples$Library.Prep.Lot
    x$samples$Library.Prep.Lot[prep_lot!=2] <- 999
    keep.exprs <- filterByExpr( x, group=group)
    x <- x[keep.exprs,, keep.lib.sizes=FALSE]
    # split cellranger gene ENSEMBL and SYMBOLS 
    comb_geneids <- rownames(x)
    split_nms <- sapply( comb_geneids, function(x) {strsplit( x, "--", fixed=TRUE)[[1]]}[c(1,2)])
    # re-format and label name types
    geneid <- t(split_nms)
    colnames(geneid) <- c("ENSEMBL","SYMBOL")
    row.names(geneid) <- 1:nrow(geneid)
    rownames(x) <- geneid[,2]
    x$genes <- geneid[,1]
    # TMM normalize
    x <- calcNormFactors(x, method ="TMM") # "TMM"
    lcpm <- cpm(x, log=TRUE)
    # create design matrix
    chem <- as.factor( x$samples$chem)
    sex  <- as.factor( x$samples$Sex)
    lot  <- as.factor( x$samples$Library.Prep.Lot)
    nlvls = c( nlevels( chem), nlevels( sex), nlevels( lot))
    factors <- c( "+ chem","+ sex","+ lot")
    des_form = as.formula( paste("~0 + group", paste( factors[nlvls>1], collapse="")))
    design <- model.matrix( des_form)
               
    colnames(design) <- gsub("group", "", colnames(design))
    colnames(design) <- make.names( colnames( design))
    ### need to be sure to only make comparison of stages present in data
    # pull stage_ids in design matrix
    contr_nms <- colnames( design)
    # pull intersect with stage_order
    inter_nms <- intersect( make.names( obs$stage_id), contr_nms)
    # create contrast vector for comparisons across stages
    num_nms <- length( inter_nms)
    # empty vector to hold contrast comparisons
    comp_vec <- c()
    # append comparisons
    for (itr in 1:num_nms){
        itr_nms2 <- inter_nms[-c(1:itr)]
        itr_nms1 <- rep( c(inter_nms[itr]), times=length(itr_nms2))
        comp_vec <- append( comp_vec, paste( itr_nms1, itr_nms2, sep='-'))
    }
    contr.matrix <- makeContrasts( contrasts=comp_vec, levels=colnames( design))
    v <- voom(x, design, plot=FALSE)
    vfit <- lmFit(v, design)
    vfit <- contrasts.fit(vfit, contrasts=contr.matrix)
    efit <- eBayes(vfit)
    tfit <- treat( vfit, lfc=1)
    dt <- decideTests(efit)
    
    iii <- 1
    for (itr in comp_vec){
        glMDPlot(efit, coef=iii, status=dt, main=colnames(efit)[iii],
             side.main="SYMBOL", counts=lcpm, groups=group, launch=FALSE, folder=dt_nm, html=itr) # html="MD-Plot"
        iii <- iii + 1
    }
    logTMM_fil_nm <- paste( dt_nm, 'logTMM_cts.csv', sep="/")
    write.csv( lcpm, logTMM_fil_nm)
    results_fl_nm <- paste( dt_nm, 'results_file.txt', sep="/")
    write.fit( efit, dt, file=results_fl_nm)
    DEG_fl_nm <- paste( dt_nm, 'DEGlist.RDS', sep="/")
    saveRDS( x, DEG_fl_nm)
}

[1] "../data/limma-voom/glimma_dev-traj_Astro"
[1] "../data/limma-voom/glimma_dev-traj_ID2"
[1] "../data/limma-voom/glimma_dev-traj_L2-3_CUX2"
[1] "../data/limma-voom/glimma_dev-traj_L4_RORB"
[1] "../data/limma-voom/glimma_dev-traj_L5-6_THEMIS"
[1] "../data/limma-voom/glimma_dev-traj_L5-6_TLE4"
[1] "../data/limma-voom/glimma_dev-traj_LAMP5_NOS1"
[1] "../data/limma-voom/glimma_dev-traj_Micro"
[1] "../data/limma-voom/glimma_dev-traj_Oligo"
[1] "../data/limma-voom/glimma_dev-traj_OPC"
[1] "../data/limma-voom/glimma_dev-traj_Poor-Quality"
Coefficients not estimable: Fetal Neonatal 


Warning message:
“Partial NA coefficients for 2752 probe(s)”


Coefficients not estimable: Fetal Neonatal 


Warning message:
“Partial NA coefficients for 2752 probe(s)”


[1] "../data/limma-voom/glimma_dev-traj_PV"
[1] "../data/limma-voom/glimma_dev-traj_PV_SCUBE3"
[1] "../data/limma-voom/glimma_dev-traj_SST"
[1] "../data/limma-voom/glimma_dev-traj_Vas"
[1] "../data/limma-voom/glimma_dev-traj_VIP"
